## Libraries

In [1]:
%%capture
! pip install pandas scikit-learn keybert transformers torch huggingface_hub keyphrase-vectorizers

In [ ]:
# Standard library imports
import os
import re
import time
import gc
import itertools
import warnings
import configparser

# Third-party imports
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import torch

# Hugging Face imports
from huggingface_hub import login
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# KeyBERT and Sentence Transformers imports
from keybert import KeyBERT, KeyLLM
from keybert.llm import TextGeneration
from keyphrase_vectorizers import KeyphraseCountVectorizer
from sentence_transformers import SentenceTransformer

## Set-Up

In [ ]:
# Load Hugging Face Token
config = configparser.ConfigParser()
config.read('./config.ini')
HF_TOKEN = config['hf_token']['access_token'] # For Hugging Face
login(HF_TOKEN)

In [4]:
warnings.simplefilter('ignore')

## Load SemEval data

In [10]:
!ls data/SemEval2017

docsutf8     keys         lan.txt      language.txt


In [ ]:
root_doc= 'data/SemEval2017/docsutf8/'
root_key= 'data/SemEval2017/keys/'

In [7]:
# Creazione di una lista per salvare i dati
data = []

# Scansione della cartella per leggere i file .txt
for filename in os.listdir(root_doc):
    if filename.endswith(".txt"):  # Controlla che sia un file di testo
        file_path = os.path.join(root_doc, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        data.append({"ID": re.sub('.txt', '', filename), "text": content})

# Creazione del DataFrame
doc_df = pd.DataFrame(data)
doc_df.head()

,ID,text
0,S0370269304009062,The microwave background is not the only unive...
1,S0375960115005630,The systems in which the Stern–Gerlach force i...
2,S0301932215002037,There is also a lack of agreement as to what c...
3,S0032386109001463,When incompatible three component polymer chai...
4,S0022311513010313,"The four bounding PCM wastes, given in Table 1..."


In [8]:
# Creazione di una lista per salvare i dati
data = []

# Scansione della cartella per leggere i file .txt
for filename in os.listdir(root_key):
    if filename.endswith(".key"):  # Controlla che sia un file di testo
        file_path = os.path.join(root_key, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read() # sep '\n'
        data.append({"ID": re.sub('.key', '', filename), "keys": content})

# Creazione del DataFrame
key_df = pd.DataFrame(data)
key_df.head()

,ID,keys
0,S0963869514001066,conventional TFM\ndiscretely sampled\ndiscreti...
1,S0167931713002438,a-SiO2\na-SiO2 leading\nbulk\ndeep electron tr...
2,S0963869514001078,electromagnetic ultrasound generation\nEMATs\n...
3,S0038092X14000942,ambient temperature\ndirect solar radiation in...
4,S0304399111001811,angular divergence\naperture\ncreation of nm-s...


In [9]:
sem_eval_df = pd.merge(doc_df, key_df, on="ID")
sem_eval_df.head()

,ID,text,keys
0,S0370269304009062,The microwave background is not the only unive...,cosmological evolution of UHECR injection\nflu...
1,S0375960115005630,The systems in which the Stern–Gerlach force i...,density waves\nelectromagnetic field\nelectrom...
2,S0301932215002037,There is also a lack of agreement as to what c...,churn\nchurn flow\ndisturbance waves\nfrequenc...
3,S0032386109001463,When incompatible three component polymer chai...,chain entropy\ncylinders\ndiagonal bond method...
4,S0022311513010313,"The four bounding PCM wastes, given in Table 1...",aluminium\nblast-furnace slag “Calumite”\nboun...


## Test different KeyWords approaches

In [10]:
def compute_metrics(pred_kw, true_kw):
    """
    Computes precision, recall, and F1-score for keyword/keyphrase extraction.

    Args:
        pred_kw (list of list): Nested lists with predicted keywords/keyphrases.
        true_kw (list of list): Nested lists with true keywords/keyphrases.

    Returns:
        dict: Dictionary with evaluation metrics (precision, recall, F1-score).
    """
    precision_list, recall_list, f1_list = [], [], []

    for pred, true in zip(pred_kw, true_kw):
        pred_set, true_set = set(pred), set(true)  # Convert to sets

        # Create binary vectors for sklearn
        all_keywords = list(true_set | pred_set)  # Union of all keywords
        y_true = [1 if kw in true_set else 0 for kw in all_keywords]
        y_pred = [1 if kw in pred_set else 0 for kw in all_keywords]

        # Calculate metrics with sklearn
        precision_list.append(precision_score(y_true, y_pred, zero_division=0))
        recall_list.append(recall_score(y_true, y_pred, zero_division=0))
        f1_list.append(f1_score(y_true, y_pred, zero_division=0))

    # Average the metrics
    eval_dict = {
        "precision": sum(precision_list) / len(precision_list),
        "recall": sum(recall_list) / len(recall_list),
        "f1_score": sum(f1_list) / len(f1_list),
    }

    return eval_dict

In [ ]:
KEY_LLM_PROMPT = """
<s>[INST] <<SYS>>

You are a helpful assistant specialized in extracting comma-separated keywords.
You are to the point and only give the answer in isolation without any chat-based fluff.

<</SYS>>
I have the following document:
- The website mentions that it only takes a couple of days to deliver but I still have not received mine.

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST] meat, beef, eat, eating, emissions, steak, food, health, processed, chicken [INST]

I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST]
"""

def initialize_models(embedding_model='all-MiniLM-L6-v2', llm_model='gpt-2', use_keyllm=True):
    """
    Loads and initializes machine learning models for embeddings, keyword extraction, text generation, 
    question answering, and summarization, using GPU if available.

    Args:
        embedding_model (str): Model for sentence embeddings. Defaults to 'all-MiniLM-L6-v2'.
        llm_model (str): Model for text generation. Defaults to 'gpt-2'.
        qa_model (str): Model for question answering. Defaults to 'distilbert-base-cased-distilled-squad'.
        sum_model (str): Model for summarization. Defaults to 'facebook/bart-large-cnn'.

    Returns:
        tuple: Initialized models:
            - SentenceTransformer for embeddings.
            - KeyBERT for keyword extraction.
            - KeyLLM for LLM-based keyword extraction.
            - HuggingFace pipelines for question answering and summarization.
    """
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    try:
        # Initialize the sentence transformer model
        print("Loading Sentence Transformer model...")
        model = SentenceTransformer(embedding_model, device=device)
        
        # Initialize the KeyBERT model
        print("Loading KeyBERT model...")
        kw_bert_model = KeyBERT(model)
        
        # Initialize the KeyLLM model
        if use_keyllm:
            print("Loading KeyLLM model...")
            tokenizer = AutoTokenizer.from_pretrained(llm_model)
            tokenizer.pad_token = tokenizer.eos_token
            llm_model = AutoModelForCausalLM.from_pretrained(
                llm_model,
                trust_remote_code=True,
                device_map='auto'
            )
            generator = pipeline(
                model=llm_model, tokenizer=tokenizer,
                task='text-generation',
                max_new_tokens=50,
                repetition_penalty=1.1,
                model_kwargs={"load_in_4bit": True}
            )
            llm = TextGeneration(generator, prompt=KEY_LLM_PROMPT)  # KEYLLM_PROMPT global variable
            kw_llm_model = KeyLLM(llm)
        else:
            kw_llm_model = None
        
        print("Models loaded successfully!")
    except Exception as e:
        print(f"An error occurred while loading the models: {e}")
    
    return model, kw_bert_model, kw_llm_model


def extract_keywords_from_text(docs, kw_bert_model, kw_llm_model, use_keyllm, diversity, top_n):
    """
    Extracts keywords from the input document(s) using either KeyBERT or a combination of KeyBERT and KeyLLM.

    Args:
        docs (str or list): The input text(s) to process.
        kw_bert_model (KeyBERT): A KeyBERT model for keyword extraction.
        kw_llm_model (KeyLLM): A KeyLLM model for refining keywords (used if `use_keyllm` is True).
        use_keyllm (bool): If True, uses KeyLLM to refine the keywords extracted by KeyBERT.
        diversity (float): Diversity parameter for MMR to ensure varied keywords.
        top_n (int): Number of top keywords to return.

    Returns:
        list: A list of extracted keywords.
    """
    
    if use_keyllm:
        # Extract initial keywords with KeyBERT (limit to 20 for LLM processing)
        initial_keywords = kw_bert_model.extract_keywords(
            docs=docs, 
            vectorizer=KeyphraseCountVectorizer(), 
            use_mmr=True, 
            diversity=diversity, 
            top_n=20
        )
        
        # Refine the keywords using KeyLLM
        refined_keywords = kw_llm_model.extract_keywords(docs=docs, candidate_keywords=initial_keywords)
        initial_keywords = [key[0] for key in initial_keywords]
        
        # Concatenation candidates from KeyBert and KeyLLM
        final_candidates = list(set(initial_keywords) | set(refined_keywords[0])) 

        # Re-rank the refined keywords and select the top_n with KeyBERT
        keywords_list = kw_bert_model.extract_keywords(
            docs=docs, 
            candidates=final_candidates,
            use_mmr=True, 
            diversity=diversity, 
            top_n=top_n # < 20
        )
    else:
        # Directly extract the top_n keywords using KeyBERT
        keywords_list = kw_bert_model.extract_keywords(
            docs=docs, 
            vectorizer=KeyphraseCountVectorizer(), 
            use_mmr=False, #True
            diversity=diversity, 
            top_n=top_n
        )
    keywords_list = [key[0] for key in keywords_list]
    return keywords_list


def test_keywords_extraction(kw_bert_model, kw_llm_model, texts, true_kw, use_keyllm, diversity, top_n):
    """
    Tests keyword extraction with KeyBERT or KeyBERT + LLM, calculating execution time and metrics.

    Args:
        kw_bert_model: KeyBERT model for keyword extraction.
        kw_llm_model: LLM model for refining keywords (used only if use_keyllm=True).
        texts (list of str): List of texts to process.
        true_kw (list of list): List of reference keywords.
        use_keyllm (bool): If True, use KeyBERT + LLM. If False, use only KeyBERT.
        diversity (float): Diversity parameter for MMR.
        top_n (int): Maximum number of keywords to extract per text.

    Returns:
        dict: Dictionary with execution time and precision, recall, and F1-score metrics.
    """
    pred_kw = []
    start_time = time.time()  # Start time measurement

    # Generate keywords for each text
    for text in tqdm(texts, desc="Processing texts"):
        keywords_list = extract_keywords_from_text(text, kw_bert_model, kw_llm_model, use_keyllm, diversity, top_n)
        pred_kw.append(keywords_list)

    end_time = time.time()  # End time measurement

    # Calculate performance metrics
    eval_metrics = compute_metrics(pred_kw, true_kw)
    eval_metrics["execution_time"] = end_time - start_time

    return eval_metrics



In [13]:

def grid_search_extraction(param_combinations, texts, true_kw):
    """
    Performs a Grid Search to test different parameter combinations
    in the test_extraction function, freeing GPU memory between iterations.

    Args:
        param_combinations (list of tuples): List with all parameter combinations.
        texts (list of str): List of texts to process.
        true_kw (list of list): List of reference keywords.

    Returns:
        pd.DataFrame: DataFrame with metric results for each combination.
    """
    results = []

    # Test each parameter combination
    for embedding_model, llm_model, use_keyllm, diversity, top_n in tqdm(param_combinations, desc="Processing parameter combinations"):

        print(f"Testing: emb_model={embedding_model}, llm_model={llm_model}, use_keyllm={use_keyllm}, diversity={diversity}, top_n={top_n}")

        # Initialize the KeyBERT model with the specified embedding model
        _, kw_bert_model, kw_llm_model = initialize_models(embedding_model, llm_model, use_keyllm)

        # Start the test with the current parameters
        metrics = test_keywords_extraction(
            kw_bert_model, kw_llm_model, texts, true_kw,
            use_keyllm=use_keyllm, diversity=diversity, top_n=top_n
        )

        # Save the results in a list
        results.append({
            "embedding_model": embedding_model,
            "llm_model": llm_model,
            "use_keyllm": use_keyllm,
            "diversity": diversity,
            "top_n": top_n,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1_score": metrics["f1_score"],
            "execution_time": metrics["execution_time"]
        })

        del kw_bert_model
        del kw_llm_model
        gc.collect()
        torch.cuda.empty_cache()

    # Convert the results to a DataFrame for easier analysis
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:

# Define parameters for Grid Search
param_grid = {
    "embedding_model": ["all-MiniLM-L6-v2"],#, "paraphrase-MiniLM-L12-v2"],  # KeyBERT embedding models
    "llm_model": ["meta-llama/Llama-3.2-3B"],  # LLM models for KeyLLM
    "use_keyllm": [True, False],  # Test both KeyBERT and KeyBERT + LLM
    "diversity": [0.3, 0.5, 0.7],  # Variation of the MMR parameter
    "top_n": [3, 5, 10]  # Maximum number of extracted keywords
}

# Generate all possible parameter combinations
param_combinations = list(itertools.product(
    param_grid["embedding_model"],
    param_grid["llm_model"],
    param_grid["use_keyllm"],
    param_grid["diversity"],
    param_grid["top_n"]
))

In [ ]:
texts = sem_eval_df["text"].tolist()
true_kw = sem_eval_df["keys"].str.split("\n").tolist()

# Test della funzione grid_search_extraction
results_df = grid_search_extraction(param_combinations, texts, true_kw)
print("Grid Search Results:")
print(results_df)

In [ ]:
# pick the best model
best_f1_params = results_df.iloc[results_df["f1_score"].idxmax()]
best_precision_params = results_df.iloc[results_df["precision"].idxmax()]
best_recall_params = results_df.iloc[results_df["recall"].idxmax()]

# print the best model parameters
print("Best F1 Score Parameters:")
print(best_f1_params)
print("\nBest Precision Parameters:")
print(best_precision_params)
print("\nBest Recall Parameters:")
print(best_recall_params)